Training the Megadetector on the COD10K Dataset: This notebook was able to complete the training process but opun seeing the results we realised that we made quite a few mistakes in the dataset preparation, which is why the results were not ideal.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/dataset

/content/drive/MyDrive/dataset


In [ ]:
!git clone https://github.com/microsoft/CameraTraps.git

Cloning into 'CameraTraps'...
remote: Enumerating objects: 19526, done.
remote: Counting objects: 100% (342/342), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 19526 (delta 137), reused 131 (delta 122), pack-reused 19184 (from 4)
Receiving objects: 100% (19526/19526), 496.77 MiB | 16.60 MiB/s, done.
Resolving deltas: 100% (10190/10190), done.
Updating files: 100% (166/166), done.


In [ ]:
%cd CameraTraps/PW_FT_detection

/content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection


In [ ]:
import os, shutil, json
from PIL import Image

# --- Adjust this to where your COD10K folder lives ---
BASE = '/content/drive/MyDrive/dataset/cod10k/COD10K-v3'

# Raw COCO JSONs & images
TRAIN_JSON = os.path.join(BASE, 'Train/CAM_Instance_Train.json')
TEST_JSON  = os.path.join(BASE, 'Test/CAM_Instance_Test.json')
TRAIN_IMG  = os.path.join(BASE, 'Train/Image')
TEST_IMG   = os.path.join(BASE, 'Test/Image')

# Where YOLO expects data
YOLO_ROOT = '/content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/COD10K'
for sub in ('images/train','labels/train','images/test','labels/test'):
    os.makedirs(os.path.join(YOLO_ROOT, sub), exist_ok=True)


In [ ]:
!ls /content/drive/MyDrive/dataset/cod10k/COD10K-v3/Train

CAM_Instance_Train.json        GT_Edge	    GT_Object
CAM-NonCAM_Instance_Train.txt  GT_Instance  Image


In [ ]:
# 1) Copy all training .jpgs
for fn in os.listdir(TRAIN_IMG):
    if fn.lower().endswith('.jpg'):
        shutil.copy(os.path.join(TRAIN_IMG,fn),
                    os.path.join(YOLO_ROOT,'images/train',fn))

# 2) Load COCO JSON
with open(TRAIN_JSON) as f:
    coco = json.load(f)

# 3) Build lookup and group bboxes
img_meta  = {img['id']:(img['file_name'],img['width'],img['height']) for img in coco['images']}
ann_by_id = {}
for ann in coco['annotations']:
    ann_by_id.setdefault(ann['image_id'],[]).append(ann['bbox'])

# 4) Write YOLO .txt labels
for img_id, bbs in ann_by_id.items():
    fn, W, H = img_meta[img_id]
    lines = []
    for x,y,w,h in bbs:
        xc = (x + w/2)/W; yc = (y + h/2)/H
        wn = w/W;      hn = h/H
        lines.append(f"0 {xc:.6f} {yc:.6f} {wn:.6f} {hn:.6f}")
    out = os.path.join(YOLO_ROOT,'labels/train',fn.replace('.jpg','.txt'))
    with open(out,'w') as f: f.write("\n".join(lines))


In [ ]:
# 1) Copy all test .jpgs
for fn in os.listdir(TEST_IMG):
    if fn.lower().endswith('.jpg'):
        shutil.copy(os.path.join(TEST_IMG,fn),
                    os.path.join(YOLO_ROOT,'images/test',fn))

# 2) Load COCO JSON
with open(TEST_JSON) as f:
    coco = json.load(f)

# 3) Build lookup
img_meta  = {img['id']:(img['file_name'],img['width'],img['height']) for img in coco['images']}
ann_by_id = {}
for ann in coco['annotations']:
    ann_by_id.setdefault(ann['image_id'],[]).append(ann['bbox'])

# 4) Write YOLO .txt labels
for img_id, bbs in ann_by_id.items():
    fn, W, H = img_meta[img_id]
    lines = []
    for x,y,w,h in bbs:
        xc = (x + w/2)/W; yc = (y + h/2)/H
        wn = w/W;      hn = h/H
        lines.append(f"0 {xc:.6f} {yc:.6f} {wn:.6f} {hn:.6f}")
    out = os.path.join(YOLO_ROOT,'labels/test',fn.replace('.jpg','.txt'))
    with open(out,'w') as f: f.write("\n".join(lines))


In [ ]:
%%bash
cat > /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/COD10K.yaml <<EOF
path: ../data/COD10K
train: images/train
val:   images/test
test:  images/test
names: ['foreground']
EOF

echo "Wrote data/COD10K.yaml:"
cat /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/COD10K.yaml


Wrote data/COD10K.yaml:
path: ../data/COD10K
train: images/train
val:   images/test
test:  images/test
names: ['foreground']


In [ ]:
%%bash
# move into your PW_FT_detection folder
cd /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of yolov5 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.1 MB/s eta 

In [ ]:
#this one is to run it first so we get the weights
%cd /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection
!python main.py --task validation --config config.yaml

/content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection
YOLOv9e summary: 721 layers, 58,147,225 parameters, 0 gradients, 192.7 GFLOPs
Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9e summary (fused): 279 layers, 57,378,713 parameters, 0 gradients, 189.1 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.2 ms, read: 0.1±0.0 MB/s, size: 92.0 KB)
val: Scanning /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/data_example/labels/train... 10 images, 0 backgrounds, 0 corrupt: 100% 10/10 [00:13<00:00,  1.34s/it]
val: New cache created: /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/data_example/labels/train.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:00<00:00,  1.04it/s]
                   all         10         10      0.993          1      0.995      0.754
                animal         10         10      0.993          1      0.995      0.754
Speed: 0.7ms 

this is my config.yaml for the run above:
# General
model: YOLO #YOLO or RTDETR
model_name: MDV6-yolov9-e
data: /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/data_example.yaml
test_data: /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/data_example/images/val
task: validation
exp_name: MDV6-yolov9-e

# Train
epochs: 20
batch_size_train: 16
imgsz: 640
device_train: 0
workers: 8
optimizer: auto
lr0: 0.01
patience: 5
save_period: 1
val: True
resume: False
weights: None # Path to weight to resume training

# Validation
save_json: True
plots: True
device_val: 0
batch_size_val: 12

and my data_example.yaml:
names:
  0: animal
  1: person
  2: vehicle
path: /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/data_example
test: images/val
train: images/train
val:   images/val

# Ok now starts the fun part

In [ ]:
# === General ===
model: YOLO
model_name: MDV6-yolov9-c
data: ./data/COD10K.yaml
task: train
exp_name: COD10K_finetune

# === Training ===
epochs: 20
batch_size_train: 16
imgsz: 640
device_train: 0
workers: 8
optimizer: auto
lr0: 0.01
patience: 5       # <-- already present
save_period: 1    # <-- make sure this line is here
val: True
resume: False
weights: None

# === Validation (during training) ===
save_json: True
plot: True
device_val: 0
batch_size_val: 12

current config for all the stuf going on

SyntaxError: invalid syntax (<ipython-input-10-70dca45d6676>, line 5)

In [ ]:
%cd /content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection

# start fine-tuning
!python main.py --task train --config config.yaml

/content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection
YOLOv9c summary: 358 layers, 25,531,545 parameters, 0 gradients, 103.7 GFLOPs
Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/torch/hub/checkpoints/MDV6b-yolov9c.pt, data=/content/drive/MyDrive/dataset/CameraTraps/PW_FT_detection/data/COD10K.yaml, epochs=20, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=0, workers=8, project=runs/train_COD10K_finetune, name=exp2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False,

In [ ]:
runs/train_COD10K_finetune/exp2/weights/
 ├─ best.pt   ← highest mAP checkpoint
 └─ last.pt   ← final-epoch checkpoint

In [ ]:
switch these in config.yaml:
task: validation
resume: True
weights: runs/train_COD10K_finetune/exp2/weights/best.pt

SyntaxError: invalid syntax (<ipython-input-14-ea44cdf095ea>, line 1)

In [ ]:
cd PW_FT_detection
python main.py --task validation --config config.yaml

In [ ]:
switch these in config.yaml:
task: inference
test_data: data/COD10K/images/infer
weights: runs/train_COD10K_finetune/exp2/weights/best.pt

In [ ]:
python main.py --task inference --config config.yaml